In [ ]:
!pip install langchain
!pip install accelerate
!pip install langchain_core
!pip install langchain_community
!pip install pypdfium2
!pip langchain-chroma
!pip install python-docx


In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

In [ ]:
! wget https://huggingface.co/QuantFactory/NeuralDaredevil-8B-abliterated-GGUF/resolve/main/NeuralDaredevil-8B-abliterated.Q8_0.gguf

In [12]:
from langchain_community.document_loaders.pdf import PyPDFium2Loader as pdf

loader = pdf("book2.pdf")
pages = loader.load()
print(pages)


text=" "
for i in range(0,len(pages)):
  text=text+pages[i].page_content

In [ ]:
from docx import Document

def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

file_path = 'output.docx'
text = read_docx(file_path)

print(text)


In [ ]:
!pip install langchain-chroma

In [13]:
#Text Splitter and Embeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import CharacterTextSplitter
import re
import langchain
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain_text_splitters import SpacyTextSplitter


def text_splitter_And_Similarity_Search(documents):



    model_name="BAAI/bge-large-en-v1.5"
    model_kwargs = {"device": "cpu","trust_remote_code":True}
    encode_kwargs = {"normalize_embeddings": True}



    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=250,
    #                                                    chunk_overlap=50)




    embedding_function = SentenceTransformerEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    )

    db = Chroma.from_documents(documents, embedding_function,persist_directory="./embeddings" )


    # embedding_function = SentenceTransformerEmbeddings(model_name="thenlper/gte-base")
    # db = Chroma.from_documents(documents, embedding_function )

    return db,







In [12]:
model_name="BAAI/bge-large-en-v1.5"
model_kwargs = {"device": "cpu","trust_remote_code":True}
encode_kwargs = {"normalize_embeddings": True}



    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=250,
    #                                                    chunk_overlap=50)




embedding_function = SentenceTransformerEmbeddings(
model_name=model_name,
model_kwargs=model_kwargs,
encode_kwargs=encode_kwargs,
)

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

def getDocs(text):
  docs=Document(page_content=text,metadata={"source":"book"})
  doc=[]
  doc.append(docs)
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                                       chunk_overlap=0)
  documents = text_splitter.split_documents(doc)
  return documents

In [15]:
db=text_splitter_And_Similarity_Search(getDocs(text))

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def check_token_limit(new_documents):
  tokenizer = AutoTokenizer.from_pretrained("mlabonne/NeuralDaredevil-8B-abliterated")
  tokens=tokenizer(new_documents)
  n_tokens=len(tokens.input_ids)
  return n_tokens

In [17]:
vectordb = Chroma(persist_directory="embeddings",embedding_function=embedding_function)
vectordb

In [44]:
k=100
def search(prompt):
  docs = vectordb.similarity_search_with_score(query=prompt,k=k)
  print("Documents:")
  print(len(docs))
  print("::Documents")
  new_documents=" "
  for i in range(0,len(docs)):
    temp_doc=docs[i][0].page_content
    print("Chunk : : \n : "+ temp_doc)
    check_doc=new_documents+temp_doc
    tokens=check_token_limit(check_doc)
    if tokens>650:
      doc=temp_doc
      for x in range(0,5):
        doc= doc[:len(doc) // 2]
        temp_tokens=check_token_limit(doc+new_documents)
        if temp_tokens<=600:
          new_documents =new_documents+doc + "\n\n\n"
          break


      print(tokens)
      return new_documents
    else:
      new_documents=check_doc + "\n\n\n"

  print(tokens)
  return new_documents

In [19]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate


callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [20]:
n_gpu_layers = -1
n_batch = 4096  # Adjust the batch size to a smaller value

llm = LlamaCpp(
    model_path="/content/NeuralDaredevil-8B-abliterated.Q8_0.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    verbose=True,
    n_ctx=4096,
    # n_tokens=3000,
    # n_ubatch=3000,
)


llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/NeuralDaredevil-8B-abliterated.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head_

In [ ]:
# # llm_chain = prompt | llm
# question = "Who is the prime minister of Pakistan?"
# llm_chain.invoke({"question": question})

In [ ]:

# check_doc= check_doc[:len(check_doc) // 2]
# # print(len(new_documents))
# new_documents = new_documents[:len(new_documents) // 2]


directly or indirectly CONTROLS the CONTRACTOR (including any intermediate or ultimate parent \ncompany) . \n Without prejudice to the CONTRACTOR ’s other rights or remedies, the CONTRACTOR may terminate \nthe AGREEMENT in the circumstances described in Articles  23.7(a)(ii)(B) .    \nADNOC -CICV -505A(v 6) 84 insert the reference numbe r \n ADNOC Classification: Need -To-Know  \n The AGREEMENT may not be terminated or otherwise ended by either PARTY on any ground, or for \nany reason, except as set forth in the AGREEMENT.  The PARTIES hereby expressly exclude, to the \nfullest extent permitted by LAW, the application of Articles  267 through 275 of the CIVIL CODE as \nwell as Articles  877, 886 and 892 through 896 of the CIVIL CODE.  For the avoidance of doubt, except \nas set forth in the AGREEMENT, the PARTIES hereby expressly exclude any requirements of notice, \nproportionality, or of the need to have recourse to a court or tribunal that may be provided for in the \nCIVIL CODE.COD

In [ ]:
questions = [
    "Describe the contractor liabilities.",
    "Which clauses cover claims from Contractor?",
    "Under which clauses Contractor can claim extension of time?",
    "Under which clauses Contractor can claim extension in call off completion date?",
    "Under which clauses Contractor can claim adjustment of Fees?",
    "What events constitute a variation?",
    "Which clauses consider Contractor's claims to Company?",
    "Which clauses describe Company's claims and back charge to Contractor?",
    "Which clauses cover claims from Company?"
]


In [ ]:
for prompt in questions:
  new_documents=search(prompt)
  question="You have been given a legal document , answer the question form that document"+" legal document "+new_documents+  " question  "+prompt

  llm.invoke(question)
